# LAR integrals

* ### Paolo Di Simone 584638
* ### Filippo Gaglioti 582704
* ### Federico Pasqui 501749

## API:
* *function M(alpha::Int, beta::Int)::Float64*
* *function TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)*
* *function II(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt=false)::Float64*
* *function III(P::LAR, alpha::Int, beta::Int, gamma::Int)::Float64*
* *function surface(P::LAR, signedInt::Bool=false)::Float64*
* *function volume(P::LAR)::Float64*
* *function firstMoment(P::LAR)::Array{Float64,1}*
* *function secondMoment(P::LAR)::Array{Float64,1}*
* *function inertiaProduct(P::LAR)::Array{Float64,1}*
* *function centroid(P::LAR)::Array{Float64,1}*
* *function inertiaMoment(P::LAR)::Array{Float64,1}*
* *function chainAreas(V::Array{Float64,2},EV::Array{Int64,2},chains::Array{Int64,2})* (**non sviluppata in questo notebook**)
* *function chainAreas(V::Array{Float64,2}, EV::Array{Int64,2}, chains::Array{Array{Int64,1},1})* (**non sviluppata in questo notebook**)

## Librerie importate e costanti

In [ ]:
using LinearAlgebra

In [ ]:
const Points = Matrix
const Cells = Array{Array{Int,1},1}
const LAR = Union{ Tuple{Points, Cells},Tuple{Points, Cells, Cells} }

## Codice base

In [ ]:
function M(alpha::Int, beta::Int)::Float64
    a = 0
    for l=0:(alpha + 1)
        a += binomial(alpha+1,l) * (-1)^l/(l+beta+1)
    end
    return a/(alpha + 1)
end

La funzione `TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)` è la funzione
principale di integrazione, essa permette di calcolare la superficie di un singolo triangolo.

In [ ]:
function TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)
	vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
	a = va - vo
	b = vb - vo
	s1 = 0.0
	for h=0:alpha
		for k=0:beta
			for m=0:gamma
				s2 = 0.0
				for i=0:h 
					s3 = 0.0
					for j=0:k
						s4 = 0.0
						for l=0:m
							s4 += binomial(m,l) * a[3]^(m-l) * b[3]^l * M( 
								h+k+m-i-j-l, i+j+l )
						end
						s3 += binomial(k,j) * a[2]^(k-j) * b[2]^j * s4
					end
					s2 += binomial(h,i) * a[1]^(h-i) * b[1]^i * s3;
				end
				s1 += binomial(alpha,h) * binomial(beta,k) * binomial(gamma,m) * 			
						vo[1]^(alpha-h) * vo[2]^(beta-k) * vo[3]^(gamma-m) * s2
			end
		end
	end
	c = cross(a,b)
	if signedInt == true
		return s1 * norm(c) * sign(c[3])
	else
		return s1 * norm(c)
	end	
end

Le funzioni `II(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt=false)::Float64` e 
`III(P::LAR, alpha::Int, beta::Int, gamma::Int)::Float64` si occupano di calcolare l'area di ogni triangolo
che forma l'oggetto (rappresentato secondo notazione LAR) passato in input `P::LAR`. Si noti quindi che l'oggetto in input deve essere triangolato.

In [ ]:
function II(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt=false)::Float64
    w = 0
    V, FV = P
    if typeof(FV) == Array{Int64,2}
    	FV = [FV[:,k] for k=1:size(FV,2)]
    end
    for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        if size(tau,2) == 3
        	term = TT(tau, alpha, beta, gamma, signedInt)
        	if signedInt
        		w += term
        	else
        		w += abs(term)
        	end
        elseif size(tau,2) > 3
        	println("ERROR: FV[$(i)] is not a triangle")
        else
        	println("ERROR: FV[$(i)] is degenerate")
        end
    end    
    return w
end


In [ ]:
function III(P::LAR, alpha::Int, beta::Int, gamma::Int)::Float64
    w = 0
    V, FV = P
    for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
        a = va - vo
        b = vb - vo
        c = cross(a,b)
        w += c[1]/norm(c) * TT(tau, alpha+1, beta, gamma)
    end
    return w/(alpha + 1)
end

Le restanti funzioni permettono di calcolare:
* Superficie: `surface(P::Lar.LAR, signedInt::Bool=false)::Float64`
* Volume: `volume(P::LAR)::Float64`
* Momento primo: `firstMoment(P::LAR)::Array{Float64,1}`
* Momento secondo: `secondMoment(P::LAR)::Array{Float64,1}`
* Prodotto d'inerzia: `inertiaProduct(P::LAR)::Array{Float64,1}`
* Baricentro: `centroid(P::LAR)::Array{Float64,1}`
* Momento d'inerzia: `inertiaMoment(P::LAR)::Array{Float64,1}`

In [ ]:
function surface(P::Lar.LAR, signedInt::Bool=false)::Float64
    return II(P, 0, 0, 0, signedInt)
end

In [ ]:
function volume(P::LAR)::Float64
    return III(P, 0, 0, 0)
end

In [ ]:
function firstMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    out[1] = III(P, 1, 0, 0)
    out[2] = III(P, 0, 1, 0)
    out[3] = III(P, 0, 0, 1)
    return out
end

In [ ]:
function secondMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    out[1] = III(P, 2, 0, 0)
    out[2] = III(P, 0, 2, 0)
    out[3] = III(P, 0, 0, 2)
    return out
end

In [ ]:
function inertiaProduct(P::LAR)::Array{Float64,1}
    out = zeros(3)
    out[1] = III(P, 0, 1, 1)
    out[2] = III(P, 1, 0, 1)
    out[3] = III(P, 1, 1, 0)
    return out
end

In [ ]:
function centroid(P::LAR)::Array{Float64,1}
	return firstMoment(P)./volume(P)
end

In [ ]:
function inertiaMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    result = secondMoment(P)
    out[1] = result[2] + result[3]
    out[2] = result[3] + result[1]
    out[3] = result[1] + result[2]
    return out
end

## Benchmark

In [ ]:
using LinearAlgebraicRepresentation
using Plasm
using BenchmarkTools
Lar = LinearAlgebraicRepresentation

Di seguito si riporta la funzione contenuta nel modulo https://github.com/cvdlab/LinearAlgebraicRepresentation.jl/blob/master/src/utilities.jl dal momento che c'era un errore nel codice e quindi l'oggetto restituito non era corretto.

In [ ]:
function obj2lar(path)
    vs = Array{Float64, 2}(undef, 0, 3)
    edges = Array{Array{Array{Int, 1}, 1}, 1}()
    faces = Array{Array{Array{Int, 1}, 1}, 1}()
    push!(edges, Array{Array{Int, 1}, 1}[])
    push!(faces, Array{Array{Int, 1}, 1}[])
    g = 1

    open(path, "r") do fd
        for line in eachline(fd)
            elems = split(line)
            if length(elems) > 0
                if elems[1] == "v"
                    
                    x = parse(Float64, elems[2])
                    y = parse(Float64, elems[3])
                    z = parse(Float64, elems[4])

                    vs = [vs; x y z]
                    
                elseif elems[1] == "f"  
                    
                    v1 = parse(Int, split(elems[2], "/")[1])
                    v2 = parse(Int, split(elems[3], "/")[1])
                    v3 = parse(Int, split(elems[4], "/")[1])
                    
                    e1 = sort([v1, v2])
                    e2 = sort([v2, v3])
                    e3 = sort([v3, v1])
                    

                    push!(edges[g], e1)
                    push!(edges[g], e2)
                    push!(edges[g], e3)

                    push!(faces[g], sort([v1, v2, v3]))

                elseif elems[1] == "g"

                    g += 1
                    push!(edges, Array{Array{Int, 1}, 1}[])
                    push!(faces, Array{Array{Int, 1}, 1}[])
                    
                end
            end
        end
    end

    return convert(Lar.Points, vs'), edges[1:end], faces[1:end]
end

In [ ]:
V, EV, FV = obj2lar("stanford-bunny.obj")
EV = EV[1]
FV = FV[1]
VV = [[k] for k=1:size(V,2)]
model = (V, (VV, EV, FV))
P = V, FV

In [ ]:
Plasm.view(V, FV)

Calcolando il volume dello *Stanford Bunny* possiamo osservare che questo viene negativo, questo perché le funzioni
`III(P::LAR, alpha::Int, beta::Int, gamma::Int)::Float64` e `volume(P::LAR)::Float64` non permettono di calcolare l'integrale senza segno. Opportune correzioni verranno fatte successivamente.

In [ ]:
volume(P)

In [ ]:
@benchmark volume(P)